<a href="https://colab.research.google.com/github/zb15/B2BNetworkWiki/blob/main/Function2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## FUNCTION 2. get_companies_network(QIDs, num_runs=5,5,5,5)

Currently the package has 3 main functions:            

1.   choose_company(input_name, search_option='all')
2.   get_companies_network(QIDs, num_runs=(5,5,5,5))
3.   visualise_b2b_network(df)

The **choose_company()** allows user to search companies with various search options: exact match; starts with the input plus space; starts with the input plus comma plus space, or as default all options.
User can search and choose any number of companies, and will need to save the QIDs only to a list to use them in the next function.



### Function to get parent info

In [ ]:
# Checking if entity has a parent, and if so how many
def has_parent(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT ?parent (IF (BOUND(?parent), COUNT(DISTINCT ?parent), 0) AS ?count)
          WHERE {{
                VALUES ?item {{wd:{items}}}.
                ?item wdt:P31 ?type .
                VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                            wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                            wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                            wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities
                ?article schema:about ?item .
                ?article schema:inLanguage "en" .
                ?article schema:isPartOf <https://en.wikipedia.org/>.

                OPTIONAL {{ ?item wdt:P749 ?parent. }}

                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                GROUP BY ?parent
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()
    # Extracting all the results
    results = data["results"]["bindings"]
    # Check if results is empty
    if not results:
      # Return zero
      return 0
    else:
      # Get the count value
      counts = results[0]["count"]["value"]
      # Make counts integers
      counts = int(counts)
      # Return the count as an integer
      return counts





In [ ]:
# Defining a function that takes a Wikidata ID and returns a dictionary of information about its industry, parent organizations and subsidiaries
def get_company_parent_info(wikidata_id):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f"""
    SELECT DISTINCT ?item
                    (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                    ?itemLabel
                    ?itemcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?industryLabel; separator="; ") AS ?industries)
                    ?parent
                    (REPLACE(STR(?parent), "http://www.wikidata.org/entity/", "") AS ?pQID)
                    ?parentLabel
                    ?parentcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?pindustryLabel; separator="; ") AS ?pindustries)
                    ?proportion
                    ?proportionofLabel
                    ?pointoftime
                    ?starttime
                    ?endtime
    WHERE {{
      VALUES ?item {{wd:{wikidata_id}}}.
      ?item wdt:P31 ?type .
      ?article schema:about ?item .
      ?article schema:inLanguage "en" .
      ?article schema:isPartOf <https://en.wikipedia.org/>.

        OPTIONAL {{ ?item rdfs:label ?itemLabel. FILTER(LANG(?itemLabel) = "en") }}
        OPTIONAL {{ ?item     wdt:P17    ?itemcountry .
                    ?itemcountry  rdfs:label  ?itemcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?itemcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item     wdt:P452    ?industry .
                    ?industry  rdfs:label  ?industryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?industryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item wdt:P749 ?parent. }}
        OPTIONAL {{ ?parent     wdt:P452    ?pindustry .
                    ?pindustry  rdfs:label  ?pindustryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?pindustryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?parent     wdt:P17    ?parentcountry .
                    ?parentcountry  rdfs:label  ?parentcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?parentcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item  p:P749 [ps:P749 ?parent; pq:P1107 ?proportion] }}
        OPTIONAL {{
                  ?item  p:P749 [ps:P749 ?parent; pq:P642 ?proportionof] .
                  FILTER(?proportionof = wd:Q144368)
                 }}
        OPTIONAL {{ ?item p:P749 [ps:P749 ?parent; pq:P585 ?pointoftime] }}
        OPTIONAL {{
                  ?item p:P749 [ps:P749 ?parent; pq:P585 ?pointoftime] .
                  FILTER(BOUND(?pointoftime) && DATATYPE(?pointoftime) = xsd:dateTime).
                  # get the latest record first
                  BIND(NOW() - ?pointoftime AS ?distance).
                  FILTER (MIN (?distance)) .
                 }}
        OPTIONAL {{ ?item  p:P749 [ps:P749 ?parent; pq:P580 ?starttime] }}
        OPTIONAL {{ ?item  p:P749 [ps:P749 ?parent; pq:P582 ?endtime] }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY  ?item ?QID ?itemLabel ?itemcountryLabel ?parent ?pQID ?parentLabel ?parentcountryLabel ?proportion ?proportionofLabel ?pointoftime ?starttime ?endtime
    ORDER BY ?parentLabel
    LIMIT 10000
    """

    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # defining url
    url = 'https://query.wikidata.org/sparql'
    # Sending the request to the Wikidata endpoint with the user agent header
    r = requests.get(url, params={'format': 'json', 'query': query}, headers={"User-Agent": user_agent})
    # extracts the results and bindings keys from the JSON response returned by the Wikidata SPARQL endpoint
    data = r.json()['results']['bindings']
    # converts the JSON data into a pandas DataFrame
    df = pd.json_normalize(data)
    #  checks if an element is a dictionary and if so, extracts the value key from it
    df = df.applymap(lambda x: x['value'] if isinstance(x, dict) else x)
    # selects only the columns that end with .value.
    df = df[[col for col in df.columns if col.endswith('.value')]]
    # removes the .value suffix from each column name
    df.columns = [col.split('.')[0] for col in df.columns]

    return df


### Function to get owned by info

In [ ]:
# Checking if entity has a ownedby (which is also a type of company), and if so how many
def has_ownedby(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT ?ownedby (IF (BOUND(?ownedby), COUNT(DISTINCT ?ownedby), 0) AS ?count)
          WHERE {{
                VALUES ?item {{wd:{items}}}.
                ?item wdt:P31 ?type .
                VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                            wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                            wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                            wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities
                ?article schema:about ?item .
                ?article schema:inLanguage "en" .
                ?article schema:isPartOf <https://en.wikipedia.org/>.

                OPTIONAL {{ ?item wdt:P127 ?ownedby. }}

                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                GROUP BY ?ownedby
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()
    # Extracting all the results
    results = data["results"]["bindings"]
    # Check if results is empty
    if not results:
      # Return zero
      return 0
    else:
      # Get the count value
      counts = results[0]["count"]["value"]
      # Make counts integers
      counts = int(counts)
      # Return the count as an integer
      return counts





In [ ]:
# Defining a function that takes a Wikidata ID and returns a dictionary of information about its industry, parent organizations and subsidiaries
def get_company_ownedby_info(wikidata_id):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f"""
    SELECT DISTINCT ?item
                    (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                    ?itemLabel
                    ?itemcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?industryLabel; separator="; ") AS ?industries)
                    ?ownedby
                    (REPLACE(STR(?ownedby), "http://www.wikidata.org/entity/", "") AS ?obQID)
                    ?ownedbyLabel
                    ?ownedbycountryLabel
                    (GROUP_CONCAT ( DISTINCT ?obindustryLabel; separator="; ") AS ?obindustries)
                    ?proportion
                    ?proportionofLabel
                    ?pointoftime
                    ?starttime
                    ?endtime
    WHERE {{
      VALUES ?item {{wd:{wikidata_id}}}.
      ?item wdt:P31 ?type .
      ?article schema:about ?item .
      ?article schema:inLanguage "en" .
      ?article schema:isPartOf <https://en.wikipedia.org/>.

        OPTIONAL {{ ?item rdfs:label ?itemLabel. FILTER(LANG(?itemLabel) = "en") }}
        OPTIONAL {{ ?item     wdt:P17    ?itemcountry .
                    ?itemcountry  rdfs:label  ?itemcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?itemcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item     wdt:P452    ?industry .
                    ?industry  rdfs:label  ?industryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?industryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item wdt:P127 ?ownedby. }}
        OPTIONAL {{ ?ownedby     wdt:P452    ?obindustry .
                    ?obindustry  rdfs:label  ?obindustryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?obindustryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?ownedby     wdt:P17    ?ownedbycountry .
                    ?ownedbycountry  rdfs:label  ?ownedbycountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?ownedbycountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item  p:P127 [ps:P127 ?ownedby; pq:P1107 ?proportion] }}
        OPTIONAL {{
                  ?item  p:P127 [ps:P127 ?ownedby; pq:P642 ?proportionof] .
                  FILTER(?proportionof = wd:Q144368)
                 }}
        OPTIONAL {{ ?item p:P127 [ps:P127 ?ownedby; pq:P585 ?pointoftime] }}
        OPTIONAL {{
                  ?item p:P127 [ps:P127 ?ownedby; pq:P585 ?pointoftime] .
                  FILTER(BOUND(?pointoftime) && DATATYPE(?pointoftime) = xsd:dateTime).
                  # get the latest record first
                  BIND(NOW() - ?pointoftime AS ?distance).
                  FILTER (MIN (?distance)) .
                 }}
        OPTIONAL {{ ?item  p:P127 [ps:P127 ?ownedby; pq:P580 ?starttime] }}
        OPTIONAL {{ ?item  p:P127 [ps:P127 ?ownedby; pq:P582 ?endtime] }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY  ?item ?QID ?itemLabel ?itemcountryLabel ?ownedby ?obQID ?ownedbyLabel ?ownedbycountryLabel ?proportion ?proportionofLabel ?pointoftime ?starttime ?endtime
    ORDER BY ?ownedbyLabel
    LIMIT 10000
    """

     # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # defining url
    url = 'https://query.wikidata.org/sparql'
    # Sending the request to the Wikidata endpoint with the user agent header
    r = requests.get(url, params={'format': 'json', 'query': query}, headers={"User-Agent": user_agent})
    # extracts the results and bindings keys from the JSON response returned by the Wikidata SPARQL endpoint
    data = r.json()['results']['bindings']
    # converts the JSON data into a pandas DataFrame
    df = pd.json_normalize(data)
    #  checks if an element is a dictionary and if so, extracts the value key from it
    df = df.applymap(lambda x: x['value'] if isinstance(x, dict) else x)
    # selects only the columns that end with .value.
    df = df[[col for col in df.columns if col.endswith('.value')]]
    # removes the .value suffix from each column name
    df.columns = [col.split('.')[0] for col in df.columns]

    return df


### Function to get subsidiary info

In [ ]:
# Checking if entity has a subsidiary (which is also a type of company), and if so how many
def has_subsidiary(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT ?subsidiary (IF (BOUND(?subsidiary), COUNT(DISTINCT ?subsidiary), 0) AS ?count)
          WHERE {{
                VALUES ?item {{wd:{items}}}.
                ?item wdt:P31 ?type .
                VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                            wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                            wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                            wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities
                ?article schema:about ?item .
                ?article schema:inLanguage "en" .
                ?article schema:isPartOf <https://en.wikipedia.org/>.

                OPTIONAL {{ ?item wdt:P355 ?subsidiary. }}

                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                GROUP BY ?subsidiary
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()
    # Extracting all the results
    results = data["results"]["bindings"]
    # Check if results is empty
    if not results:
      # Return zero
      return 0
    else:
      # Get the count value
      counts = results[0]["count"]["value"]
      # Make counts integers
      counts = int(counts)
      # Return the count as an integer
      return counts





In [ ]:
# getting company's subsidaries
def get_company_subsidiary_info(wikidata_id):
    import requests
    import pandas as pd
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    query = f"""
        SELECT DISTINCT ?item
                        (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                        ?itemLabel
                        ?itemcountryLabel
                        (GROUP_CONCAT ( DISTINCT ?industryLabel; separator="; ") AS ?industries)
                        ?subsidiary
                        (REPLACE(STR(?subsidiary), "http://www.wikidata.org/entity/", "") AS ?sQID)
                        ?subsidiaryLabel
                        ?subsidiarycountryLabel
                        (GROUP_CONCAT ( DISTINCT ?sindustryLabel; separator="; ") AS ?sindustries)
                        ?proportion
                        ?proportionofLabel
                        ?pointoftime
                        ?starttime
                        ?endtime
        WHERE {{
          VALUES ?item {{wd:{wikidata_id}}}.
          ?item wdt:P31 ?type .
          ?article schema:about ?item .
          ?article schema:inLanguage "en" .
          ?article schema:isPartOf <https://en.wikipedia.org/>.

            OPTIONAL {{ ?item rdfs:label ?itemLabel. FILTER(LANG(?itemLabel) = "en") }}
            OPTIONAL {{ ?item     wdt:P17    ?itemcountry .
                    ?itemcountry  rdfs:label  ?itemcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?itemcountryLabel ), "en" ) )
                  }}
            OPTIONAL {{ ?item     wdt:P452    ?industry .
                        ?industry  rdfs:label  ?industryLabel
                        FILTER ( LANGMATCHES ( LANG ( ?industryLabel ), "en" ) )
                      }}
            OPTIONAL {{ ?item wdt:P355 ?subsidiary. }}
            OPTIONAL {{ ?subsidiary     wdt:P452    ?sindustry .
                        ?sindustry  rdfs:label  ?sindustryLabel
                        FILTER ( LANGMATCHES ( LANG ( ?sindustryLabel ), "en" ) )
                      }}
            OPTIONAL {{ ?subsidiary     wdt:P17    ?subsidiarycountry .
                    ?subsidiarycountry  rdfs:label  ?subsidiarycountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?subsidiarycountryLabel ), "en" ) )
                  }}
            OPTIONAL {{ ?item  p:P355 [ps:P355 ?subsidiary; pq:P1107 ?proportion] }}
            OPTIONAL {{
                        ?item  p:P355 [ps:P355 ?subsidiary; pq:P642 ?proportionof] .
                        FILTER(?proportionof = wd:Q144368)
                     }}
            OPTIONAL {{ ?item p:P355 [ps:P355 ?subsidiary; pq:P585 ?pointoftime] }}
            OPTIONAL {{
                        ?item p:P355 [ps:P355 ?subsidiary; pq:P585 ?pointoftime] .
                        FILTER(BOUND(?pointoftime) && DATATYPE(?pointoftime) = xsd:dateTime).
                        # get the latest record first
                        BIND(NOW() - ?pointoftime AS ?distance).
                        FILTER (MIN (?distance)) .
                      }}
            OPTIONAL {{ ?item  p:P355 [ps:P355 ?subsidiary; pq:P580 ?starttime] }}
            OPTIONAL {{ ?item  p:P355 [ps:P355 ?subsidiary; pq:P582 ?endtime] }}
          SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        GROUP BY  ?item ?QID ?itemLabel ?itemcountryLabel ?subsidiary ?sQID ?subsidiaryLabel ?subsidiarycountryLabel ?proportion ?proportionofLabel ?pointoftime ?starttime ?endtime
        ORDER BY ?subsidiaryLabel
        LIMIT 10000
        """

    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # defining url
    url = 'https://query.wikidata.org/sparql'
    # Sending the request to the Wikidata endpoint with the user agent header
    r = requests.get(url, params={'format': 'json', 'query': query}, headers={"User-Agent": user_agent})
    # extracts the results and bindings keys from the JSON response returned by the Wikidata SPARQL endpoint
    data = r.json()['results']['bindings']
    # converts the JSON data into a pandas DataFrame
    df = pd.json_normalize(data)
    #  checks if an element is a dictionary and if so, extracts the value key from it
    df = df.applymap(lambda x: x['value'] if isinstance(x, dict) else x)
    # selects only the columns that end with .value.
    df = df[[col for col in df.columns if col.endswith('.value')]]
    # removes the .value suffix from each column name
    df.columns = [col.split('.')[0] for col in df.columns]

    return df

### Function to get owner of info

In [ ]:
# Checking if entity has a ownerof (which is also a type of company), and if so how many
def has_ownerof(items):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f""" SELECT ?ownerof (IF (BOUND(?ownerof), COUNT(DISTINCT ?ownerof), 0) AS ?count)
          WHERE {{
                VALUES ?item {{wd:{items}}}.
                ?item wdt:P31 ?type .
                VALUES ?type {{wd:Q4830453 wd:Q783794 wd:Q6881511 wd:Q167037 wd:Q21980538 wd:Q891723 wd:Q786820 wd:Q43229 wd:Q1058914
                            wd:Q18388277 wd:Q161726 wd:Q778575 wd:Q2005696 wd:Q108460239 wd:Q3477381 wd:Q270791 wd:Q936518
                            wd:Q1934969 wd:Q2538889 wd:Q2995256 wd:Q1631129 wd:Q1276157 wd:Q5038204 wd:Q217107 wd:Q13235160
                            wd:Q17377208 wd:Q740752 wd:Q249556}} . #to search multiple entities
                ?article schema:about ?item .
                ?article schema:inLanguage "en" .
                ?article schema:isPartOf <https://en.wikipedia.org/>.

                OPTIONAL {{ ?item wdt:P1830 ?ownerof. }}

                SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
                }}
                GROUP BY ?ownerof
    """
    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # Sending the request to the Wikidata endpoint with the user agent header
    response = requests.get("https://query.wikidata.org/sparql", params={"query": query, "format": "json"}, headers={"User-Agent": user_agent})
    # Parsing the response as JSON
    data = response.json()
    # Extracting all the results
    results = data["results"]["bindings"]
    # Check if results is empty
    if not results:
      # Return zero
      return 0
    else:
      # Get the count value
      counts = results[0]["count"]["value"]
      # Make counts integers
      counts = int(counts)
      # Return the count as an integer
      return counts





In [ ]:
# Defining a function that takes a Wikidata ID and returns a dictionary of information about its industry, parent organizations and subsidiaries
def get_company_ownerof_info(wikidata_id):
    # Defining a list of user agents to alternate
    user_agents = [
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Safari/605.1.15",
        "Mozilla/5.0 (X11; Linux x86_64; rv:95.0) Gecko/20100101 Firefox/95.0",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.2 Mobile/15E148 Safari/604.1"
    ]
    # Constructing the SPARQL query
    query = f"""
    SELECT DISTINCT ?item
                    (REPLACE(STR(?item), "http://www.wikidata.org/entity/", "") AS ?QID)
                    ?itemLabel
                    ?itemcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?industryLabel; separator="; ") AS ?industries)
                    ?ownerof
                    (REPLACE(STR(?ownerof), "http://www.wikidata.org/entity/", "") AS ?ooQID)
                    ?ownerofLabel
                    ?ownerofcountryLabel
                    (GROUP_CONCAT ( DISTINCT ?ooindustryLabel; separator="; ") AS ?ooindustries)
                    ?proportion
                    ?proportionofLabel
                    ?pointoftime
                    ?starttime
                    ?endtime
    WHERE {{
      VALUES ?item {{wd:{wikidata_id}}}.
      ?item wdt:P31 ?type .
      ?article schema:about ?item .
      ?article schema:inLanguage "en" .
      ?article schema:isPartOf <https://en.wikipedia.org/>.

        OPTIONAL {{ ?item rdfs:label ?companyLabel. FILTER(LANG(?companyLabel) = "en") }}
        OPTIONAL {{ ?item     wdt:P17    ?itemcountry .
                    ?itemcountry  rdfs:label  ?itemcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?itemcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item     wdt:P452    ?industry .
                    ?industry  rdfs:label  ?industryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?industryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item wdt:P1830 ?ownerof. }}
        OPTIONAL {{ ?ownerof     wdt:P452    ?ooindustry .
                    ?ooindustry  rdfs:label  ?ooindustryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?ooindustryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?ownerof     wdt:P17    ?ownerofcountry .
                    ?ownerofcountry  rdfs:label  ?ownerofcountryLabel
                    FILTER ( LANGMATCHES ( LANG ( ?ownerofcountryLabel ), "en" ) )
                  }}
        OPTIONAL {{ ?item  p:P1830 [ps:P1830 ?ownerof; pq:P1107 ?proportion] }}
        OPTIONAL {{
                    ?item  p:P1830 [ps:P1830 ?ownerof; pq:P642 ?proportionof] .
                    FILTER(?proportionof = wd:Q144368)
                  }}
        OPTIONAL {{ ?item p:P1830 [ps:P1830 ?ownerof; pq:P585 ?pointoftime] }}
        OPTIONAL {{
                    ?item p:P1830 [ps:P1830 ?ownerof; pq:P585 ?pointoftime] .
                    FILTER(BOUND(?pointoftime) && DATATYPE(?pointoftime) = xsd:dateTime).
                    # get the latest record first
                    BIND(NOW() - ?pointoftime AS ?distance).
                    FILTER (MIN (?distance)) .
                  }}
        OPTIONAL {{ ?item  p:P1830 [ps:P1830 ?ownerof; pq:P580 ?starttime] }}
        OPTIONAL {{ ?item  p:P1830 [ps:P1830 ?ownerof; pq:P582 ?endtime] }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
    }}
    GROUP BY  ?item ?QID ?itemLabel ?itemcountryLabel ?ownerof ?ooQID ?ownerofLabel ?ownerofcountryLabel ?proportion ?proportionofLabel ?pointoftime ?starttime ?endtime
    ORDER BY ?ownerofLabel
    LIMIT 10000
    """

    # Choosing a random user agent from the list
    user_agent = random.choice(user_agents)
    # defining url
    url = 'https://query.wikidata.org/sparql'
    # Sending the request to the Wikidata endpoint with the user agent header
    r = requests.get(url, params={'format': 'json', 'query': query}, headers={"User-Agent": user_agent})
    # extracts the results and bindings keys from the JSON response returned by the Wikidata SPARQL endpoint
    data = r.json()['results']['bindings']
    # converts the JSON data into a pandas DataFrame
    df = pd.json_normalize(data)
    #  checks if an element is a dictionary and if so, extracts the value key from it
    df = df.applymap(lambda x: x['value'] if isinstance(x, dict) else x)
    # selects only the columns that end with .value.
    df = df[[col for col in df.columns if col.endswith('.value')]]
    # removes the .value suffix from each column name
    df.columns = [col.split('.')[0] for col in df.columns]

    return df

### Define the main function

In [ ]:
def tidy_each_df(df):
  #split companies' industries string and return them as list, then drop duplicates, and finally return as string again
  df.iloc[:, 4] = df.iloc[:,4].str.split("; ").apply(lambda x: "; ".join(set(word.lower() for word in x)))
  # drop parents' industries duplicates, then return as string again
  df.iloc[:, 9] = df.iloc[:, 9].str.split("; ").apply(lambda x: "; ".join(set(word.lower() for word in x)))
  # check if all proprtions are between 0 and 1, if above 1, then devide by 100, otherwise leave it
  if 'proportion' in df.columns:
    df['proportion'] = df['proportion'].astype(float)
    df['proportion'] = np.where(df['proportion'].isnull(), df['proportion'],
                                np.where(df['proportion'] > 1, df['proportion']/100,
                                         df['proportion']))
  df = df.drop_duplicates().reset_index(drop=True)
  return df

In [ ]:
# companies_network function

# define the main function
def get_companies_network(ids, levels=(5,5,5,5)):
  import numpy as np
  import pandas as pd
  # Create an empty dataframe for parent
  p_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'parent', \
                               'pQID', 'parentLabel', 'parentcountryLabel', 'pindustries', 'proportion', \
                               'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  # Create an empty dataframe for owned by
  ob_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'ownedby', \
                                'obQID', 'ownedbyLabel', 'ownedbycountryLabel', 'obindustries', 'proportion', \
                                'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  # Create an empty dataframe for subsidiary
  s_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'subsidiary', \
                               'sQID', 'subsidiaryLabel', 'subsidiarycountryLabel', 'sindustries', 'proportion', \
                               'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])
  # Create an empty dataframe for owner of
  oo_df = pd.DataFrame(columns=['item', 'QID', 'itemLabel', 'itemcountryLabel', 'industries', 'ownerof', \
                                'ooQID', 'ownerofLabel', 'ownerofcountryLabel', 'ooindustries', 'proportion', \
                                'proportionofLabel', 'pointoftime', 'starttime', 'endtime'])

  # initialize a list to store the results of each run
  results = {"p_df": [], "ob_df": [], "s_df": [], "oo_df": []}
  # making sure only those QIDs in root_QIDs that starts with "Q"
  root_QIDs = [item for item in ids if item.startswith("Q")]
  # Find the maximum number of levels
  max_levels = max(levels)
  # Iterate from 0 to max_levels-1
  for j in range(max_levels):
    # Iterate over each element of the levels tuple
    for i in range(len(levels)):
      # Check if j is less than the value of levels[i]
      if j < levels[i]:
          # Iterate over each element of the item list
          if i == 0:
            # making sure only those QIDs in root_QIDs that starts with "Q"
            root_QIDs = [item for item in root_QIDs if item.startswith("Q")]
            ## HAS PARENT
            # make a new parent QID list with only those that has parent
            new_p_QIDs = []
            for id in root_QIDs:
              parent_count = has_parent(id)
              if parent_count > 0:
                new_p_QIDs.append(id)
            ## PARENT INFO
            # Check if new_p_QIDs is empty
            # get the parent info from the new QID list and concat it with the p_df dataframe
            for id in new_p_QIDs:
              p_df = pd.concat([p_df, get_company_parent_info(id)], ignore_index=True)
            # tidy up industries and proportions with function tidy_each_df()
            p_df = tidy_each_df(p_df)
          elif i == 1:
            # making sure only those QIDs in root_QIDs that starts with "Q"
            root_QIDs = [item for item in root_QIDs if item.startswith("Q")]
            ## HAS OWNED BY
            # make a new ownedby QID list with only those companies that has ownedby
            new_ob_QIDs = []
            for id in root_QIDs:
              ownedby_count = has_ownedby(id)
              if ownedby_count > 0:
                new_ob_QIDs.append(id)
            ## GET OWNED BY INFO
            for id in new_ob_QIDs:
              ob_df = pd.concat([ob_df, get_company_ownedby_info(id)], ignore_index=True)
            # tidy up industries and proportions with function tidy_each_df()
            ob_df = tidy_each_df(ob_df)
          elif i == 2:
            # making sure only those QIDs in root_QIDs that starts with "Q"
            root_QIDs = [item for item in root_QIDs if item.startswith("Q")]
            ## HAS SUBSIDIARY
            # make a new subsidiary QID list with only those companies that has subsidiary
            new_s_QIDs = []
            for id in root_QIDs:
              subsidiary_count = has_subsidiary(id)
              if subsidiary_count > 0:
                new_s_QIDs.append(id)
            ## GET SUBSIDIARY INFO
            # get the subsidiaries info from the new QID list and concat it with the s_df dataframe
            for id in new_s_QIDs:
              s_df = pd.concat([s_df, get_company_subsidiary_info(id)], ignore_index=True)
            # tidy up industries and proportions with function tidy_each_df()
            s_df = tidy_each_df(s_df)
          elif i == 3:
            # making sure only those QIDs in root_QIDs that starts with "Q"
            root_QIDs = [item for item in root_QIDs if item.startswith("Q")]
            ## HAS OWNER OF
            # make a new ownerof QID list with only those companies that has owner of
            new_oo_QIDs = []
            for id in root_QIDs:
              ownerof_count = has_ownerof(id)
              if ownerof_count > 0:
                new_oo_QIDs.append(id)
            ## GET OWNER OF INFO
            # get the owner of info from the new QID list and concat it with the oo_df dataframe
            for id in new_oo_QIDs:
              oo_df = pd.concat([oo_df, get_company_ownerof_info(id)], ignore_index=True)
            # tidy up industries and proportions with function tidy_each_df()
            oo_df = tidy_each_df(oo_df)
    # append results after each number of runs
    results['p_df'].append(p_df)
    results['ob_df'].append(ob_df)
    results['s_df'].append(s_df)
    results['oo_df'].append(oo_df)
    ## JOIN NEW UNIQUE QIDS
    #getting all the QIDs that will needs to be queried again
    # Store dataframes in a list
    df_list = [p_df, ob_df, s_df, oo_df]
    # Store column names in a list
    col_list = ['pQID', 'obQID', 'sQID', 'ooQID']
    # Create an empty list to store the column values
    val_list = []
    # Loop over the dataframes and the column names and append the column values to the list
    for df, col in zip(df_list, col_list):
      val_list.append(df[col].to_list())
    #collect the unique values from val_list
    new_list = []
    [new_list.append(x) for x in val_list if x not in new_list]
    # Convert the list to a pandas series
    original_series = pd.Series(new_list)
    # Use the explode method to split the nested lists into separate rows
    exploded_series = original_series.explode()
    # Use the drop_duplicates method to remove any duplicate values
    unique_series = exploded_series.drop_duplicates()
    # Convert the series back to a list
    new_QIDs = unique_series.tolist()
    #clean new_QIDs from 'nan' string values
    new_QIDs = [x for x in new_QIDs if str(x) != 'nan']
    # get a list of the new unique QIDs
    new1_QIDs = list(set(new_QIDs) - set(root_QIDs))
    # keep only those QIDs that starts with "Q"
    new2_QIDs = [item for item in new1_QIDs if item.startswith("Q")]
    #rename the new list of QIDs as root_QIDs
    root_QIDs = new2_QIDs

  # return the list of results
  return results